# 线性代数

In [6]:
import torch
from torch import nn
from torch.utils import data

## 生成数据

In [14]:
def synthetic_data(w,b,num_examples):
    X = torch.normal(0,1,(num_examples,len(w)))
    y = torch.matmul(X,w) + b
    y += torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1))

true_w = torch.tensor([-2,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print('features' ,features[0],'\nlabels' ,labels[0])

"""
构造一个PyTorch数据迭代器
data_arrays : 原始数组 
batch_size : 一个批次的数据数量
is_train : 设置为True时会在每个epoch重新打乱数据
"""
def load_array(data_arrays, batch_size, is_train=True):  #@save
    # 包装数据和目标张量的数据集。
    dataset = data.TensorDataset(*data_arrays)
    # 数据加载器。组合数据集和采样器，并在数据集上提供单进程或多进程迭代器。
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

print('next:',next(iter(data_iter)))
print('data_iter X:',len(next(iter(data_iter))[0]))

features tensor([ 0.2550, -0.5005]) 
labels tensor([5.3901])
next: [tensor([[-3.1823, -0.6120],
        [ 0.6100, -0.2145],
        [-0.0547, -0.1078],
        [ 0.4012, -2.3984],
        [ 0.0722,  0.6559],
        [-0.6715, -0.8784],
        [ 1.6748, -1.5111],
        [ 0.7828, -0.3389],
        [-0.8243, -0.0069],
        [ 0.0975, -0.8255]]), tensor([[12.6353],
        [ 3.6946],
        [ 4.6747],
        [11.5541],
        [ 1.8264],
        [ 8.5186],
        [ 5.9848],
        [ 3.7921],
        [ 5.8861],
        [ 6.8189]])]
data_iter X: 10


## 创建神经网络

In [9]:
# 定义一个序列神经网络
net = nn.Sequential(nn.Linear(2, 1))
print('net:',net)

# 初始化第一层线性函数的数据
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

# 创建一个标准差损失函数
loss = nn.MSELoss()

# 定义优化算法
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

# 训练
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

# 检查结果
w = net[0].weight.data
print('w:',w.reshape(true_w.shape))
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b:',b)
print('b的估计误差：', true_b - b)

net: Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
epoch 1, loss 0.000400
epoch 2, loss 0.000099
epoch 3, loss 0.000101
w: tensor([-1.9983, -3.4001])
w的估计误差： tensor([-0.0017,  0.0001])
b: tensor([4.2005])
b的估计误差： tensor([-0.0005])
